In [2]:
import numpy as np
import pandas as pd
import os
from datetime import date
import matplotlib.pyplot as plt
import time
import sys

/usr/local/lib/python3.6/site-packages/matplotlib/font_manager.py:232: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


## Q1

### Total Payment

In [11]:
series_csv = pd.Series()
file_names = []
for name in os.listdir():
    if "detail" in name:
        if "2015Q2" in name and "updated" not in name:
            continue
        file_names.append(name)
        try:
            series_csv[name[:6]] = pd.read_csv(name)
        except:
            series_csv[name[:6]] = pd.read_csv(name, encoding = 'latin1')
        
        

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
def str2float(x):
    if type(x) == str:
        x = x.replace(',', '')
        return float(x)
    return x

def str2int(x):
    x = str(x)
    if '-' in x:
        sign = -1
        x = x.replace('-', '')
    else:
        sign = 1
    x = x.replace(',', '')
    if '.' not in x:
        x = x + '.00'
    x = x.split('.')
    if len(x[1]) < 2:
        add_len = 2 - len(x[1])
        x[1] = x[1] + '0' * add_len
    x = int(x[0]) * 100 + int(x[1][:2])
    return sign*x

total_amount = []
total_amount_ = []
for item in series_csv:
    item['AMOUNT_'] = item['AMOUNT'].apply(str2int)
    item['AMOUNT'] = item['AMOUNT'].apply(str2float)
    total_amount_.append(item['AMOUNT_'])
    total_amount.append(item['AMOUNT'])
total_amount_ = pd.concat(total_amount_, ignore_index = True)
total_amount = pd.concat(total_amount, ignore_index = True)

In [13]:
print(total_amount.sum())
print(total_amount_.sum())

13660703793.3
1366070379331


### Coverage Period

In [14]:
def norm_time(time):
    time = time.split('/')
    if len(time) < 3:
        return None
    if len(time[2]) == 2:
        time[2] = '20' + time[2]
    return date(year = int(time[2]), month = int(time[0]), day = int(time[1]))


def get_days(df):
    mask = (df['START DATE'].apply(lambda x: type(x) == str and x.count('/') == 2))
    df = df[mask]
    start = df['START DATE'].apply(norm_time)   
    end = df['END DATE'].apply(norm_time) 
    return (end - start).apply(lambda x: x.days), start, end

In [15]:
periods = []
for name, item in series_csv.items():
    print(name)
    item['Coverage Period'], item['START DATE_'], item['END DATE_'] = get_days(item)
    mask = item['AMOUNT_'] > 0
    periods.append(item.loc[mask, ['START DATE', 'END DATE','Coverage Period']])
periods = pd.concat(periods, ignore_index = True)

2009Q3
2009Q4
2010Q1
2010Q2
2010Q3
2010Q4
2011Q1
2011Q2
2011Q3
2011Q4
2012Q1
2012Q2
2012Q3
2012Q4
2013Q1
2013Q2
2013Q3
2013Q4
2014Q1
2014Q2
2014Q3
2014Q4
2015Q1
2015Q2
2015Q3
2015Q4
2016Q1
2016Q2
2016Q3
2016Q4
2017Q1
2017Q2
2017Q3
2017Q4
2018Q1


In [16]:
periods['Coverage Period'].std()

63.918315772301341

### JAN-01-2010 to Dec-31-2016

In [17]:
annual_amount = []
for name, item in series_csv.items():
    print(name)
    mask = (item['START DATE_'] >= date(2010, 1, 1)) & (item['START DATE_'] <= date(2016, 12, 31))
    mask &= item['AMOUNT_'] > 0
    annual_amount.append(item.loc[mask, 'AMOUNT_'])
annual_amount = pd.concat(annual_amount, ignore_index = True)

2009Q3
2009Q4
2010Q1
2010Q2
2010Q3
2010Q4
2011Q1
2011Q2
2011Q3
2011Q4
2012Q1
2012Q2
2012Q3
2012Q4
2013Q1
2013Q2
2013Q3
2013Q4
2014Q1
2014Q2
2014Q3
2014Q4
2015Q1
2015Q2
2015Q3
2015Q4
2016Q1
2016Q2
2016Q3
2016Q4
2017Q1
2017Q2
2017Q3
2017Q4
2018Q1


In [18]:
annual_amount.sum()/700

1223251095.0314286

### Highest fraction in 2016

In [40]:
def office_name_norm(x):
    idx  = x.find('20')
    if idx >= 0:
        x = x[idx+5:]
    x = x.replace(',', ' ')
    x = x.replace('.', ' ')
    x = x.replace('"', ' ')
    x = x.replace('-', ' ')
    return ' '.join(x.split())

data_2016 = []
for name, item in series_csv.items():
    mask = (item['START DATE_'] >= date(2016, 1, 1)) & (item['START DATE_'] <= date(2016, 12, 31))
    item["OFFICE_"] = item['OFFICE'].apply(office_name_norm)
    data_2016.append(item.loc[mask])
data_2016 = pd.concat(data_2016, ignore_index = True, sort=False)

In [41]:
data_2016["OFFICE_"].unique()

array(['HON KEVIN BRADY', 'HON JUDY CHU', 'HON TIM WALBERG',
       'COMMITTEE ON AGRICULTURE', 'HON BLAKE FARENTHOLD',
       'HON MICHAEL G GRIMM', 'HON FRANK A LOBIONDO',
       'HON GRACE F NAPOLITANO', 'HON BETO OROURKE', 'HON THOMAS J ROONEY',
       'HON KEITH J ROTHFUS', 'HON ADAM SMITH', 'HON ADRIAN SMITH',
       'HOMELAND SECURITY', 'HON CANDICE S MILLER',
       'CHIEF ADMIN OFCR OF THE HOUSE', 'HON MARK E AMODEI',
       'HON MADELEINE Z BORDALLO', 'HON BILLY LONG', 'HON BETTY MCCOLLUM',
       'HON C A DUTCH RUPPERSBERGER', 'HON MAC THORNBERRY',
       'COMMITTEE ON JUDICIARY', 'COMMITTEE ON WAYS AND MEANS',
       'OFFICE OF THE SPEAKER', 'OFFICE OF THE MINORITY LEADER',
       'OFFICE OF THE MAJORITY WHIP', 'DEMOCRATIC CAUCUS',
       'LEGISLATIVE COUNSEL', 'HON ROBERT B ADERHOLT', 'HON PETE AGUILAR',
       'HON RICK W ALLEN', 'HON BRIAN BABIN', 'HON LOU BARLETTA',
       'HON JOE BARTON', 'HON KAREN BASS', 'HON JOYCE BEATTY',
       'HON XAVIER BECARRA', 'HON DAN BENI

In [42]:
offices = data_2016.groupby('OFFICE_')['AMOUNT_'].sum()
largest_office = offices.idxmax()

mask = data_2016["OFFICE_"] == largest_office
purposes = data_2016[mask].groupby('PURPOSE')['AMOUNT_'].sum()
purposes.max()/data_2016['AMOUNT_'].sum()

0.068495704125967449

### Highest average staff salary

In [43]:
representatives = []
for item in series_csv: 
    mask = item['OFFICE_'].apply(lambda x: x[:4] == 'HON ')
    names = item[mask]['OFFICE_'].unique()
    representatives.append(names)

representatives = pd.Series(np.concatenate(representatives)).unique()

In [44]:
representatives

array(['HON NEIL ABERCROMBIE', 'HON GARY L ACKERMAN',
       'HON ROBERT B ADERHOLT', 'HON JOHN H ADLER', 'HON W TODD AKIN',
       'HON RODNEY ALEXANDER', 'HON THOMAS H ALLEN', 'HON JASON ALTMIRE',
       'HON ROBERT ANDREWS', 'HON MICHAEL A ARCURI', 'HON STEVE AUSTRIA',
       'HON JOE BACA', 'HON MICHELE BACHMANN', 'HON SPENCER BACHUS',
       'HON BRIAN BAIRD', 'HON TAMMY BALDWIN', 'HON J GRESHAM BARRETT',
       'HON JOHN BARROW', 'HON ROSCOE G BARTLETT', 'HON JOE BARTON',
       'HON MELISSA L BEAN', 'HON XAVIER BECERRA', 'HON SHELLEY BERKLEY',
       'HON HOWARD L BERMAN', 'HON MARION BERRY', 'HON JUDY BIGGERT',
       'HON BRIAN BILBRAY', 'HON GUS M BILIRAKIS', 'HON ROB BISHOP',
       'HON SANFORD D BISHOP JR', 'HON TIMOTHY H BISHOP',
       'HON MARSHA BLACKBURN', 'HON EARL BLUMENAUER', 'HON ROY BLUNT',
       'HON JOHN A BOCCIERI', 'HON JOHN A BOEHNER', 'HON JO BONNER',
       'HON MARY BONO MACK', 'HON JOHN BOOZMAN', 'HON MADELINE Z BORDALLO',
       'HON DAN BOREN', 'HON L

In [45]:
def payee_norm(x):
    if type(x) != str:
        return x
    x = x.replace(',', ' ')
    x = x.replace('.', ' ')
    return ' '.join(x.split())

salaries = []
for name, item in series_csv.items():
    mask = (item['START DATE_'] >= date(2016, 1, 1)) & (item['START DATE_'] <= date(2016, 12, 31))
    mask &= item['CATEGORY'] == "PERSONNEL COMPENSATION"
    item['PAYEE_'] = item['PAYEE'].apply(payee_norm)
    salaries.append(item.loc[mask])
salaries = pd.concat(salaries, ignore_index = True, sort=False)

In [46]:
# salaries.groupby('PAYEE_')['AMOUNT'].sum()
mask = salaries['OFFICE_'].apply(lambda x: x[:4] == 'HON ')
individual_salaries = salaries[mask].groupby(['OFFICE_', "PAYEE_"])["AMOUNT_"].sum()
individual_salaries.groupby('OFFICE_').mean().max()/100


65064.538124999999

In [47]:
mask = salaries['OFFICE_'].apply(lambda x: x == 'HON STEPHEN F LYNCH')
individual_salaries = salaries[mask].groupby("PAYEE_")["AMOUNT_"].sum()
individual_salaries

PAYEE_
ADJEI-KORANTENG ARABA     3578335
BARNES SHAYNAH            5343619
CAHAN JACLYN M            6805000
FERNANDEZ BRUCE           9628335
FOWKES ROBERT J           9577915
GORDON GRETA H            3679165
GORDON JAMES P            8036665
HOLLINGSHEAD MEGAN M      3600004
KING JOSEPH T             3148889
LYNCH F D                 8337665
MURPHY JOHN P             3401250
OSORIO MARIANA T          5645415
RYAN ROBERT K            15333335
SILVA NATASHA A           3600004
ZAFERAKIS NICHOLAS        8337665
ZAPPALA ELIZABETH N       6050000
Name: AMOUNT_, dtype: int64

### Median rate of turnover

In [96]:
serve_years = pd.DataFrame(0, index = representatives, columns = list(range(2009, 2019)) + ['total'])
for year in range(2009, 2019):
    whole_year = []
    for key, item in series_csv.items():
        if key[:4] == str(year):
            whole_year.append(item[['OFFICE_', 'PAYEE_']])
    whole_year = pd.concat(whole_year, ignore_index = True)
    mask = whole_year['OFFICE_'].apply(lambda x: x[:4] == 'HON ')
    whole_year = whole_year.loc[mask]
    whole_year = whole_year.groupby('OFFICE_')['PAYEE_'].nunique()
    names = whole_year[whole_year >= 5].index
    serve_years.loc[names, 'total'] += 1
    serve_years.loc[names, year] = 1

mask = serve_years['total'] >= 4
qualified = serve_years[mask]
qualified_members = qualified.index

In [104]:
rate = pd.Series(0, list(range(2011, 2017)))
for year in range(2011, 2017):
    mask = qualified[year-1] == 1
    last_year = mask.sum()
    this_year = (qualified.loc[mask, year] == 0).sum()
    rate.loc[year] = 1.0 * this_year/last_year
rate.median()

0.016431924882629109